<a href="https://colab.research.google.com/github/leepopnamoo/SQL-Study/blob/main/Ch8_K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***군집분석***

##***데이터 불러오기***

**데이터 집합**

*   87 필드, 13,144 레코드
*   인구통계학 데이터
    *   Gender, MaritalStatus, HadChildren, …
*   거주지 데이터
    *   Population, HouseHolds, BluCollarOcc, …
*   구매 데이터
    *   RFM, TotalSpend, AveItemSpend, …



In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
respon_data = pd.read_csv("p6_cu_s.txt", delimiter=',', header=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
respon_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/빅데이터러닝센터_머신러닝&모델링_Day2/p6_cu_s.txt", delimiter=',', header=0)

##***데이터 확인***

***첫 행부터 5번째 행까지의 데이터 확인***

In [ ]:
respon_data.head(10)

***마지막 5개 행 데이터 확인***

In [ ]:
respon_data.tail(10)

***데이터 정보 확인***

In [ ]:
respon_data.info()

***데이터 요약 통계정보 확인***

In [ ]:
respon_data.describe()

##***군집 분석을 위한 데이터 선별 및 전처리***

###***구매 데이터 선별***

In [ ]:
purch_data = respon_data.iloc[:,12:26]

In [ ]:
purch_data.head()

In [ ]:
purch_data.info()

###***인구 통계학 데이터 ,거주지 데이터 선별***

In [ ]:
pop_data = respon_data.iloc[:,1:12]

In [ ]:
pop_data.head(5)

In [ ]:
pop_data.info()

###***군집 분석을 위한 데이터 전처리-인구통계 데이터***

####***Null 값 대체하기***

In [ ]:
pop_data.info()

In [ ]:
pop_data['MaritalStatus'].unique()

In [ ]:
pop_data.fillna('No answer', inplace=True)

In [ ]:
pop_data['MaritalStatus'].unique()

####***문자형 데이터 수치형으로 변환하기***

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelEncoder= LabelEncoder()
labelEncoder.fit(pop_data['HasChildren'])

In [ ]:
labelEncoder.classes_

In [ ]:
dir(labelEncoder)

In [ ]:
pop_data['HasChildren'] = labelEncoder.transform(pop_data['HasChildren'])

In [ ]:
pop_data['HasChildren'].head()

In [ ]:
pop_data['HasChildren'].unique()

####***1번째~4번째 필드 까지 문자형 데이터 수치형으로 변환하기 - 반복문***

In [ ]:
labelEncoder= LabelEncoder()
columns_index = [0, 1, 2, 3]
for i in columns_index:
    labelEncoder.fit(pop_data.iloc[:, i])
    pop_data.iloc[:, i] = labelEncoder.transform(pop_data.iloc[:, i])

In [ ]:
pop_data.info()

In [ ]:
pop_data.head()

##***군집 분석 - 인구통계 데이터***

In [ ]:
from sklearn import cluster

In [ ]:
k_means_pop = cluster.KMeans(n_clusters=3, random_state=42, verbose=1)
kmean_pop_result = k_means_pop.fit(pop_data)

In [ ]:
kmean_pop_result.labels_

In [ ]:
pop_cluster = pd.DataFrame({"cluster_label":kmean_pop_result.labels_})
pop_cluster

In [ ]:
pop_result = pd.concat([pop_data, pop_cluster, respon_data['Responded']], axis=1)
pop_result[pop_result['cluster_label']==0]

###***인구통계 데이터 군집 평가하기 - 실루엣 스코어***

In [ ]:
from sklearn import metrics
cluster_score = metrics.silhouette_score(pop_data, kmean_pop_result.labels_, metric='euclidean', sample_size=len(pop_data))
print(cluster_score)

##***군집 분석 - 구매 데이터***

In [ ]:
from sklearn import cluster

In [ ]:
k_means_purch = cluster.KMeans(n_clusters=3, random_state=42, verbose=0)
kmean_purch_result = k_means_purch.fit(purch_data)

In [ ]:
purch_cluster = pd.DataFrame({"cluster_label":kmean_purch_result.labels_})
purch_cluster

In [ ]:
purch_result = pd.concat([purch_data, purch_cluster, respon_data['Responded']], axis =1)
purch_result[purch_result['Responded']=='T']

###***구매 데이터 군집 평가하기 - 실루엣 스코어***

In [ ]:
from sklearn import metrics
cluster_score = metrics.silhouette_score(purch_data, kmean_purch_result.labels_, metric='euclidean', sample_size=len(purch_data))
print(cluster_score)

##***최적 군집 수 확인***

###***인구통계 데이터***

####***Inertia 활용***

In [ ]:
import matplotlib.pyplot as plt

number_k = range(1,10)
inertias = []

for k in number_k:
    model = cluster.KMeans(n_clusters=k)
    model.fit(pop_data)
    inertias.append(model.inertia_)
    
# Plot ks vs inertias
plt.plot(number_k, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(number_k)
plt.show()

####***Silhouette score 활용***

In [ ]:
import numpy as np

scores = []
values = np.arange(2, 10)

for num_clusters in values:
    kmeans = cluster.KMeans(n_clusters=num_clusters)
    kmeans.fit(pop_data)
    score = metrics.silhouette_score(pop_data, kmeans.labels_, 
                metric='euclidean', sample_size=len(pop_data))

    print("\nNumber of clusters =", num_clusters)
    print("Silhouette score =", score)
                    
    scores.append(score)

plt.figure()
plt.bar(values, scores, width=0.7, color='blue', align='center')
plt.title('Silhouette score vs number of clusters')

num_clusters = np.argmax(scores) + values[0]
print('\nOptimal number of clusters =', num_clusters)

plt.show()

###***구매 데이터***

####***Inertia 활용***

In [ ]:
import matplotlib.pyplot as plt

number_k = range(1,10)
inertias = []

for k in number_k:
    model = cluster.KMeans(n_clusters=k)
    model.fit(purch_data)
    inertias.append(model.inertia_)
    
# Plot ks vs inertias
plt.plot(number_k, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(number_k)
plt.show()

####***Silhouette score 활용***

In [ ]:
import numpy as np
# Initialize variables
scores = []
values = np.arange(2, 10)

# Iterate through the defined range
for num_clusters in values:
    # Train the KMeans clustering model
    kmeans = cluster.KMeans(n_clusters=num_clusters)
    kmeans.fit(purch_data)
    score = metrics.silhouette_score(purch_data, kmeans.labels_, 
                metric='euclidean', sample_size=len(purch_data))

    print("\nNumber of clusters =", num_clusters)
    print("Silhouette score =", score)
                    
    scores.append(score)

# Plot silhouette scores
plt.figure()
plt.bar(values, scores, width=0.7, color='blue', align='center')
plt.title('Silhouette score vs number of clusters')

# Extract best score and optimal number of clusters
num_clusters = np.argmax(scores) + values[0]
print('\nOptimal number of clusters =', num_clusters)

plt.show()

##***군집 분석 교차표***

###***인구통계 데이터 - 빈도로 확인***

In [ ]:
pop_result

In [ ]:
pop_cross_table = pd.crosstab(pop_result['cluster_label'], pop_result['Responded'], margins=True)
print(pop_cross_table)

####***교차표 전체 합계 확인***

In [ ]:
pop_cross_table = pd.crosstab(pop_result['cluster_label'], pop_result['Responded'], margins=True, normalize=True)
print(pop_cross_table)

###***구매 데이터 - 비율로 확인***

####***구매 데이터 군집 분석 - 5개***

In [ ]:
k_means_purch = cluster.KMeans(n_clusters=5, random_state=42, verbose=0)
n_kmean_purch_result = k_means_purch.fit(purch_data)
n_purch_cluster = pd.DataFrame({"cluster_label":n_kmean_purch_result.labels_})
n_purch_result = pd.concat([purch_data, n_purch_cluster, respon_data['Responded']], axis =1)

####***교차표 전체 합계  기준 % 확인***

In [ ]:
purch_cross_table = pd.crosstab(n_purch_result['cluster_label'], n_purch_result['Responded'], margins=True, normalize=True)
print (purch_cross_table)

####***교차표 행별 합계  기준 % 확인***

In [ ]:
pd.crosstab(n_purch_result['cluster_label'], n_purch_result['Responded'], margins=True).apply(lambda x: x/float(x[-1]), axis=1)